In [14]:
%pip install tqdm gym atari-py

Note: you may need to restart the kernel to use updated packages.


In [15]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [16]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [17]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()


State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [18]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1


print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1936
Penalties incurred: 649


In [19]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, speed=.1):
    for i, frame in enumerate(frames):
        print(frame, i, speed)
        
def print_frame(frame, i, speed):
    clear_output(wait=True)
    print(frame['frame'])
    print(f"Timestep: {i + 1}")
    print(f"State: {frame['state']}")
    print(f"Action: {frame['action']}")
    print(f"Reward: {frame['reward']}")
    sleep(speed)
# print_frames(frames)

# Q Learning

In [20]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [21]:
%%time
from tqdm import tqdm
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

def state_action_policy(state):
    if random.uniform(0, 1) < epsilon:
        return env.action_space.sample() # Explore action space
    
    return np.argmax(q_table[state]) # Exploit learned values

def learn(number_of_epochs):
    pbar = tqdm(range(number_of_epochs))

    for i in pbar:
        state = env.reset()

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            action = state_action_policy(state)

            next_state, reward, done, info = env.step(action) 

            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1

        all_epochs.append(epochs)
        all_penalties.append(penalties)
        if i % 100 == 0:
            pbar.set_description(f'Reward: {reward}, Penalties: {round(np.mean(all_penalties), 2)}, Epochs: {epochs} [Avg: {round(np.mean(all_epochs), 2)}]')


    print("Training finished.\n")


Wall time: 0 ns


In [22]:
def play():
    frames = []
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        action = state_action_policy(state)

        next_state, reward, done, info = env.step(action)
        
        print_frame({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
        }, epochs, .2)

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1


In [23]:
learn(3000)

Reward: 20, Penalties: 2.15, Epochs: 15 [Avg: 55.38]: 100%|██████████| 3000/3000 [00:03<00:00, 922.51it/s]  

Training finished.



In [24]:
play()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 222
State: 13
Action: 3
Reward: -1


KeyboardInterrupt: 

In [25]:
learn(6000)

Reward: 20, Penalties: 1.01, Epochs: 15 [Avg: 28.5]: 100%|██████████| 6000/6000 [00:02<00:00, 2519.92it/s] 

Training finished.



In [26]:
play()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 16
State: 418
Action: 5
Reward: 20


In [27]:
learn(10000)

Reward: 20, Penalties: 0.71, Epochs: 16 [Avg: 21.25]: 100%|██████████| 10000/10000 [00:03<00:00, 2654.61it/s]

Training finished.



In [29]:
play()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 10
State: 16
Action: 5
Reward: 20
